In [ ]:
#INSTALACIÓN DE LIBRERIAS
!pip install Torch==1.8.1
!pip install TorchVision

In [ ]:
#IMPORTS
import os
import shutil
import cv2

In [1]:
import os
import cv2

# Ruta de la carpeta que contiene las imágenes
ruta_carpeta = r'D:\Master\TFM\IDC_regular_ps50_idx5\00Prueba'

# Lista para almacenar los arrays de las imágenes
listado_arrays_imagenes = []
listado_arrays_etiquetas = []

# Recorrer las subcarpetas '0' y '1'
for subcarpeta in ['0', '1']:
    subcarpeta_ruta = os.path.join(ruta_carpeta, subcarpeta)

    # Para cada archivo en la subcarpeta
    for nombre_archivo in os.listdir(subcarpeta_ruta):
        # Comprobar si el archivo es una imagen (esto dependerá de las extensiones de archivo de tus imágenes)
        if nombre_archivo.endswith('.png'):
            # Construir la ruta completa del archivo
            ruta_archivo = os.path.join(subcarpeta_ruta, nombre_archivo)

            # Leer la imagen y convertirla a un array
            img = cv2.imread(ruta_archivo)
            # Añadir el array a la lista
            listado_arrays_imagenes.append(img)
            # Añadir la etiqueta correspondiente a la lista de etiquetas
            listado_arrays_etiquetas.append(int(subcarpeta))

# Ahora listado_arrays_imagenes es una lista de arrays de NumPy y listado_arrays_etiquetas contiene las etiquetas correspondientes

print(listado_arrays_imagenes[0])
print(listado_arrays_etiquetas[0])

FileNotFoundError: ignored

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Asumimos que la lista 'listado_arrays_imagenes' existe y contiene las matrices de imágenes

# Verificar la forma de las matrices
shapes = set(img.shape for img in listado_arrays_imagenes)
if len(shapes) > 1:
    # Realizar operación de ajuste en las matrices para que tengan la misma forma
    max_shape = max(shapes, key=lambda x: np.prod(x))
    listado_arrays_imagenes = [np.pad(img, [(0, max_shape[0]-img.shape[0]),
                                           (0, max_shape[1]-img.shape[1]),
                                           (0, max_shape[2]-img.shape[2])],
                                      mode='constant', constant_values=0)
                               for img in listado_arrays_imagenes]

# Convertir la lista de imágenes en una matriz 4D
imagenes = np.stack(listado_arrays_imagenes)

# Obtener las dimensiones de las imágenes
num_imagenes, altura, anchura, canales = imagenes.shape

# Reshape para que las imágenes sean 2D
imagenes_2d = imagenes.reshape(num_imagenes, -1)

# Crear una instancia del escalador MinMaxScaler
scaler = MinMaxScaler()

# Aplicar el escalado a las imágenes
imagenes_escaladas = scaler.fit_transform(imagenes_2d)

# Reshape de nuevo a las dimensiones originales de las imágenes
imagenes_escaladas = imagenes_escaladas.reshape(num_imagenes, altura, anchura, canales)


In [ ]:
print(imagenes_escaladas[0])

[[[0.80632411 0.64313725 0.88492063]
  [0.76679842 0.60392157 0.87698413]
  [0.82608696 0.68627451 0.88095238]
  ...
  [0.92941176 0.87007874 0.94117647]
  [0.83921569 0.72440945 0.90980392]
  [0.92156863 0.83858268 0.95669291]]

 [[0.73517787 0.55686275 0.84920635]
  [0.69960474 0.50980392 0.86507937]
  [0.76679842 0.58823529 0.87698413]
  ...
  [0.80314961 0.66666667 0.89370079]
  [0.84313725 0.70588235 0.89803922]
  [0.90980392 0.83137255 0.9254902 ]]

 [[0.83333333 0.70355731 0.93227092]
  [0.77865613 0.61811024 0.8968254 ]
  [0.68379447 0.49212598 0.8531746 ]
  ...
  [0.85433071 0.72440945 0.86666667]
  [0.75686275 0.6        0.74509804]
  [0.81568627 0.64566929 0.89019608]]

 ...

 [[0.75889328 0.57086614 0.85433071]
  [0.72727273 0.50588235 0.84251969]
  [0.72332016 0.50787402 0.83464567]
  ...
  [0.7254902  0.47843137 0.76078431]
  [0.75686275 0.56078431 0.8       ]
  [0.7372549  0.50588235 0.74117647]]

 [[0.75590551 0.56692913 0.85490196]
  [0.72834646 0.50196078 0.83529412]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# Convert lists to tensors
imagenes = torch.stack([torch.Tensor(img).permute(2, 0, 1) for img in imagenes])
etiquetas = torch.tensor(listado_arrays_etiquetas)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define the model architecture
class Model(nn.Module):
    def __init__(self, num_neurons):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, num_neurons, 3)
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(num_neurons*24*24, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


# Define the function to create the model
# Define the function to create the model
def crear_modelo(num_neurons):
    model = Model(num_neurons)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    return model, criterion, optimizer

# Define the grid of parameters
param_grid = {
    'num_neurons': [10, 20, 30, 40, 50],
}
# Perform grid search
best_accuracy = 0.0
best_params = None

for params in ParameterGrid(param_grid):
    model, criterion, optimizer = crear_modelo(params['num_neurons'])

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    for epoch in range(10):
        model.train()
        image_counter = 0
        for inputs, labels in train_loader:
            image_counter += inputs.size(0)
            print(f"Epoch: {epoch+1}, Processed images: {image_counter}")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                predicted = (outputs >= 0.5).squeeze().long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)


Epoch: 1, Processed images: 10
Epoch: 1, Processed images: 20
Epoch: 1, Processed images: 30
Epoch: 1, Processed images: 40
Epoch: 1, Processed images: 50
Epoch: 1, Processed images: 60
Epoch: 1, Processed images: 70
Epoch: 1, Processed images: 80
Epoch: 1, Processed images: 90
Epoch: 1, Processed images: 100
Epoch: 1, Processed images: 110
Epoch: 1, Processed images: 120
Epoch: 1, Processed images: 130
Epoch: 1, Processed images: 140
Epoch: 1, Processed images: 150
Epoch: 1, Processed images: 160
Epoch: 1, Processed images: 170
Epoch: 1, Processed images: 180
Epoch: 1, Processed images: 190
Epoch: 1, Processed images: 200
Epoch: 1, Processed images: 210
Epoch: 1, Processed images: 220
Epoch: 1, Processed images: 230
Epoch: 1, Processed images: 240
Epoch: 1, Processed images: 250
Epoch: 1, Processed images: 260
Epoch: 1, Processed images: 270
Epoch: 1, Processed images: 280
Epoch: 1, Processed images: 290
Epoch: 1, Processed images: 300
Epoch: 1, Processed images: 310
Epoch: 1, Process

Epoch: 1, Processed images: 2700
Epoch: 1, Processed images: 2710
Epoch: 1, Processed images: 2720
Epoch: 1, Processed images: 2730
Epoch: 1, Processed images: 2740
Epoch: 1, Processed images: 2750
Epoch: 1, Processed images: 2760
Epoch: 1, Processed images: 2770
Epoch: 1, Processed images: 2780
Epoch: 1, Processed images: 2790
Epoch: 1, Processed images: 2800
Epoch: 1, Processed images: 2810
Epoch: 1, Processed images: 2820
Epoch: 1, Processed images: 2830
Epoch: 1, Processed images: 2840
Epoch: 1, Processed images: 2850
Epoch: 1, Processed images: 2860
Epoch: 1, Processed images: 2870
Epoch: 1, Processed images: 2880
Epoch: 1, Processed images: 2890
Epoch: 1, Processed images: 2900
Epoch: 1, Processed images: 2910
Epoch: 1, Processed images: 2920
Epoch: 1, Processed images: 2930
Epoch: 1, Processed images: 2940
Epoch: 1, Processed images: 2950
Epoch: 1, Processed images: 2960
Epoch: 1, Processed images: 2970
Epoch: 1, Processed images: 2980
Epoch: 1, Processed images: 2990
Epoch: 1, 

Epoch: 1, Processed images: 5270
Epoch: 1, Processed images: 5280
Epoch: 1, Processed images: 5290
Epoch: 1, Processed images: 5300
Epoch: 1, Processed images: 5310
Epoch: 1, Processed images: 5320
Epoch: 1, Processed images: 5330
Epoch: 1, Processed images: 5340
Epoch: 1, Processed images: 5350
Epoch: 1, Processed images: 5360
Epoch: 1, Processed images: 5370
Epoch: 1, Processed images: 5380
Epoch: 1, Processed images: 5390
Epoch: 1, Processed images: 5400
Epoch: 1, Processed images: 5410
Epoch: 1, Processed images: 5420
Epoch: 1, Processed images: 5430
Epoch: 1, Processed images: 5440
Epoch: 1, Processed images: 5450
Epoch: 1, Processed images: 5460
Epoch: 1, Processed images: 5470
Epoch: 1, Processed images: 5480
Epoch: 1, Processed images: 5490
Epoch: 1, Processed images: 5500
Epoch: 1, Processed images: 5510
Epoch: 1, Processed images: 5520
Epoch: 1, Processed images: 5530
Epoch: 1, Processed images: 5540
Epoch: 1, Processed images: 5550
Epoch: 1, Processed images: 5560
Epoch: 1, 

Epoch: 1, Processed images: 8010
Epoch: 1, Processed images: 8020
Epoch: 1, Processed images: 8030
Epoch: 1, Processed images: 8040
Epoch: 1, Processed images: 8050
Epoch: 1, Processed images: 8060
Epoch: 1, Processed images: 8070
Epoch: 1, Processed images: 8080
Epoch: 1, Processed images: 8090
Epoch: 1, Processed images: 8100
Epoch: 1, Processed images: 8110
Epoch: 1, Processed images: 8120
Epoch: 1, Processed images: 8130
Epoch: 1, Processed images: 8140
Epoch: 1, Processed images: 8150
Epoch: 1, Processed images: 8160
Epoch: 1, Processed images: 8170
Epoch: 1, Processed images: 8180
Epoch: 1, Processed images: 8190
Epoch: 1, Processed images: 8200
Epoch: 1, Processed images: 8210
Epoch: 1, Processed images: 8220
Epoch: 1, Processed images: 8230
Epoch: 1, Processed images: 8240
Epoch: 1, Processed images: 8250
Epoch: 1, Processed images: 8260
Epoch: 1, Processed images: 8270
Epoch: 1, Processed images: 8280
Epoch: 1, Processed images: 8290
Epoch: 1, Processed images: 8300
Epoch: 1, 

Epoch: 1, Processed images: 10770
Epoch: 1, Processed images: 10780
Epoch: 1, Processed images: 10790
Epoch: 1, Processed images: 10800
Epoch: 1, Processed images: 10810
Epoch: 1, Processed images: 10820
Epoch: 1, Processed images: 10830
Epoch: 1, Processed images: 10840
Epoch: 1, Processed images: 10850
Epoch: 1, Processed images: 10860
Epoch: 1, Processed images: 10870
Epoch: 1, Processed images: 10880
Epoch: 1, Processed images: 10890
Epoch: 1, Processed images: 10900
Epoch: 1, Processed images: 10910
Epoch: 1, Processed images: 10920
Epoch: 1, Processed images: 10930
Epoch: 1, Processed images: 10940
Epoch: 1, Processed images: 10950
Epoch: 1, Processed images: 10960
Epoch: 1, Processed images: 10970
Epoch: 1, Processed images: 10980
Epoch: 1, Processed images: 10990
Epoch: 1, Processed images: 11000
Epoch: 1, Processed images: 11010
Epoch: 1, Processed images: 11020
Epoch: 1, Processed images: 11030
Epoch: 1, Processed images: 11040
Epoch: 1, Processed images: 11050
Epoch: 1, Proc

...

Epoch: 4, Processed images: 26750
Epoch: 4, Processed images: 26760
Epoch: 4, Processed images: 26770
Epoch: 4, Processed images: 26780
Epoch: 4, Processed images: 26790
Epoch: 4, Processed images: 26800
Epoch: 4, Processed images: 26810
Epoch: 4, Processed images: 26820
Epoch: 4, Processed images: 26830
Epoch: 4, Processed images: 26840
Epoch: 4, Processed images: 26850
Epoch: 4, Processed images: 26860
Epoch: 4, Processed images: 26870
Epoch: 4, Processed images: 26880
Epoch: 4, Processed images: 26890
Epoch: 4, Processed images: 26900
Epoch: 4, Processed images: 26910
Epoch: 4, Processed images: 26920
Epoch: 4, Processed images: 26930
Epoch: 4, Processed images: 26940
Epoch: 4, Processed images: 26950
Epoch: 4, Processed images: 26960
Epoch: 4, Processed images: 26970
Epoch: 4, Processed images: 26980
Epoch: 4, Processed images: 26990
Epoch: 4, Processed images: 27000
Epoch: 4, Processed images: 27010
Epoch: 4, Processed images: 27020
Epoch: 4, Processed images: 27030
Epoch: 4, Proc

Epoch: 4, Processed images: 29170
Epoch: 4, Processed images: 29180
Epoch: 4, Processed images: 29190
Epoch: 4, Processed images: 29200
Epoch: 4, Processed images: 29210
Epoch: 4, Processed images: 29220
Epoch: 4, Processed images: 29230
Epoch: 4, Processed images: 29240
Epoch: 4, Processed images: 29250
Epoch: 4, Processed images: 29260
Epoch: 4, Processed images: 29270
Epoch: 4, Processed images: 29280
Epoch: 4, Processed images: 29290
Epoch: 4, Processed images: 29300
Epoch: 4, Processed images: 29310
Epoch: 4, Processed images: 29320
Epoch: 4, Processed images: 29330
Epoch: 4, Processed images: 29340
Epoch: 4, Processed images: 29350
Epoch: 4, Processed images: 29360
Epoch: 4, Processed images: 29370
Epoch: 4, Processed images: 29380
Epoch: 4, Processed images: 29390
Epoch: 4, Processed images: 29400
Epoch: 4, Processed images: 29410
Epoch: 4, Processed images: 29420
Epoch: 4, Processed images: 29430
Epoch: 4, Processed images: 29440
Epoch: 4, Processed images: 29450
Epoch: 4, Proc

Epoch: 4, Processed images: 31940
Epoch: 4, Processed images: 31950
Epoch: 4, Processed images: 31960
Epoch: 4, Processed images: 31970
Epoch: 4, Processed images: 31980
Epoch: 4, Processed images: 31990
Epoch: 4, Processed images: 32000
Epoch: 4, Processed images: 32010
Epoch: 4, Processed images: 32020
Epoch: 4, Processed images: 32030
Epoch: 4, Processed images: 32040
Epoch: 4, Processed images: 32050
Epoch: 4, Processed images: 32060
Epoch: 4, Processed images: 32070
Epoch: 4, Processed images: 32080
Epoch: 4, Processed images: 32090
Epoch: 4, Processed images: 32100
Epoch: 4, Processed images: 32110
Epoch: 4, Processed images: 32120
Epoch: 4, Processed images: 32130
Epoch: 4, Processed images: 32140
Epoch: 4, Processed images: 32150
Epoch: 4, Processed images: 32160
Epoch: 4, Processed images: 32170
Epoch: 4, Processed images: 32180
Epoch: 4, Processed images: 32190
Epoch: 4, Processed images: 32200
Epoch: 4, Processed images: 32210
Epoch: 4, Processed images: 32220
Epoch: 4, Proc

Epoch: 4, Processed images: 34440
Epoch: 4, Processed images: 34450
Epoch: 4, Processed images: 34460
Epoch: 4, Processed images: 34470
Epoch: 4, Processed images: 34480
Epoch: 4, Processed images: 34490
Epoch: 4, Processed images: 34500
Epoch: 4, Processed images: 34510
Epoch: 4, Processed images: 34520
Epoch: 4, Processed images: 34530
Epoch: 4, Processed images: 34540
Epoch: 4, Processed images: 34550
Epoch: 4, Processed images: 34560
Epoch: 4, Processed images: 34570
Epoch: 4, Processed images: 34580
Epoch: 4, Processed images: 34590
Epoch: 4, Processed images: 34600
Epoch: 4, Processed images: 34610
Epoch: 4, Processed images: 34620
Epoch: 4, Processed images: 34630
Epoch: 4, Processed images: 34640
Epoch: 4, Processed images: 34650
Epoch: 4, Processed images: 34660
Epoch: 4, Processed images: 34670
Epoch: 4, Processed images: 34680
Epoch: 4, Processed images: 34690
Epoch: 4, Processed images: 34700
Epoch: 4, Processed images: 34710
Epoch: 4, Processed images: 34720
Epoch: 4, Proc

Epoch: 4, Processed images: 37170
Epoch: 4, Processed images: 37180
Epoch: 4, Processed images: 37190
Epoch: 4, Processed images: 37200
Epoch: 4, Processed images: 37210
Epoch: 4, Processed images: 37220
Epoch: 4, Processed images: 37230
Epoch: 4, Processed images: 37240
Epoch: 4, Processed images: 37250
Epoch: 4, Processed images: 37260
Epoch: 4, Processed images: 37270
Epoch: 4, Processed images: 37280
Epoch: 4, Processed images: 37290
Epoch: 4, Processed images: 37300
Epoch: 4, Processed images: 37310
Epoch: 4, Processed images: 37320
Epoch: 4, Processed images: 37330
Epoch: 4, Processed images: 37340
Epoch: 4, Processed images: 37350
Epoch: 4, Processed images: 37360
Epoch: 4, Processed images: 37370
Epoch: 4, Processed images: 37380
Epoch: 4, Processed images: 37390
Epoch: 4, Processed images: 37400
Epoch: 4, Processed images: 37410
Epoch: 4, Processed images: 37420
Epoch: 4, Processed images: 37430
Epoch: 4, Processed images: 37440
Epoch: 4, Processed images: 37450
Epoch: 4, Proc